In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from PIL import Image
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tqdm import tqdm

def add_padding(X):
    ### assuming X squared
    height, width = X.shape[1:3]
    new_height = int(np.ceil(height/2 * 2**0.5))*2
    new_width = int(np.ceil(width/2 * 2**0.5))*2
    offset_h = (new_height-height)//2
    offset_w = (new_width-width)//2
    top, bottom = offset_h, offset_h+(new_height+1)%2
    left, right = offset_w, offset_w+(new_width+1)%2
    paddings = tf.constant([[0,0], [top, bottom], [left, right], [0,0]])
    X = tf.pad(X, paddings, "SYMMETRIC").numpy()
    return X

def convert_to_lossless_polar(img):
    assert len(img.shape) == 3
    assert img.shape[0] == img.shape[1]
    M = img.shape[0]
    C = img.shape[-1]
    R = M//2 + 1
    W = (M+1)*4
    row, col = 0, 0
    new_img = np.zeros((R, W, C), dtype=np.float32)
    for channel in range(C):
        for r in range(R):
            flatidx = 0
            offset = 2*r + 3 
            for col in range(r, M-r-1):
                if col == r:
                    # repeat three times
                    new_img[r, flatidx:flatidx+offset-r*(r>0), channel] = img[row, col, channel]
                    flatidx += offset-r*(r>0)
                else:
                    new_img[r, flatidx, channel] = img[row, col, channel]
                    flatidx += 1
                # print(row,col)
            col += 1
            for row in range(r, M-r-1):
                if row == r:
                    # repeat three times
                    new_img[r, flatidx:flatidx+offset, channel] = img[row, col, channel]
                    flatidx += offset
                else:
                    new_img[r, flatidx, channel] = img[row, col, channel]
                    flatidx += 1
                # print(row,col)
            row += 1
            for col in range(M-r-1, r, -1):
                if col == (M-r-1):
                    # repeat three times
                    new_img[r, flatidx:flatidx+offset, channel] = img[row, col, channel]
                    flatidx += offset
                else:
                    new_img[r, flatidx, channel] = img[row, col, channel]
                    flatidx += 1
                # print(row,col)
            col -= 1
            for row in range(M-r-1, r, -1):
                if row == (M-r-1):
                    # repeat three times
                    new_img[r, flatidx:flatidx+offset, channel] = img[row, col, channel]
                    flatidx += offset
                else:
                    new_img[r, flatidx, channel] = img[row, col, channel]
                    flatidx += 1
                # print(row,col)
            # add remainder
            for e,re in enumerate(range(W-flatidx)):
                new_img[r, flatidx+re, channel] = new_img[r, e, channel]
        new_img[R-1,...,channel] = img[R-1,R-1,channel]
    return new_img

### GAUSSIAN IMAGE

In [ ]:
dummy_img = np.zeros((11, 11), dtype=np.float32)
for i in range(11):
    for j in range(11):
        dummy_img[i,j] = (i-5)**2 + (j-5)**2
dummy_img = np.exp(-0.1*dummy_img)
plt.figure()
plt.imshow(dummy_img)
plt.figure()
plt.imshow(convert_to_lossless_polar(dummy_img[...,None])[...,0])

In [3]:
def rotate(x, angle):
    # x is assumed to be in [0,1]
    xrot = np.array(Image.fromarray((x*255).astype(np.uint8)).rotate(angle)).astype(np.float32)/255.
    return xrot

def get_model(input_shape):
    in_ = Input(shape=input_shape)
    x = in_
    for i in range(3):
        x = Conv2D(2**(4+i), (3,5), strides=(1,2), padding='VALID')(x)
        x = Activation('relu')(x)
    x = Lambda(lambda t: tf.reduce_max(t, axis=2))(x)
    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation('relu')(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(in_, x)
    model.build((None,)+input_shape)
    return model

In [5]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype(np.float32)/255.
X_test  = X_test.astype(np.float32)/255.
y_train = y_train.ravel()
y_test = y_test.ravel()
real_radius = (X_train.shape[1] + (X_train.shape[1]+1)%2)//2

# new_X_train = np.load('../circle/new_X_train.npy')
# new_X_test = np.load('../circle/new_X_test.npy')
# new_X_test_45 = np.load('../circle/new_X_test_45.npy')
# new_X_test_90 = np.load('../circle/new_X_test_90.npy')

# plt.figure()
# plt.imshow(np.vstack(( new_X_test[0], new_X_test_90[0], new_X_test_45[0])))



# plt.figure()
# plt.imshow(np.vstack(( new_X_test[0], new_X_test_90[0], new_X_test_45[0])))

X_train = add_padding(X_train)
X_test = add_padding(X_test)

new_X_train = []
for x in tqdm(X_train):
    new_X_train.append(convert_to_lossless_polar(x))
    
new_X_test = []
for x in tqdm(X_test):
    new_X_test.append(convert_to_lossless_polar(x))
new_X_train = np.array(new_X_train)
new_X_test  = np.array(new_X_test)

new_X_test_rot = []
for x in tqdm(X_test):
    xr = rotate(x, 45)
    new_X_test_rot.append(convert_to_lossless_polar(xr))
new_X_test_45 = np.array(new_X_test_rot)

new_X_test_rot = []
for x in tqdm(X_test):
    xr = rotate(x, 90)
    new_X_test_rot.append(convert_to_lossless_polar(xr))
new_X_test_90 = np.array(new_X_test_rot)

offset = new_X_train.shape[1] - real_radius
new_X_train = new_X_train[:, offset:]
new_X_test = new_X_test[:, offset:]
new_X_test_45 = new_X_test_45[:, offset:]
new_X_test_90 = new_X_test_90[:, offset:]

# np.save(Cnew_X_train.npy', new_X_train)
# np.save('circle/new_X_test.npy', new_X_test)
# np.save('circle/new_X_test_45.npy', new_X_test_rot)
# np.save('circle/new_X_test_90.npy', new_X_test_rot)

170508288/170498071 [==============================] - 7s 0us/step


100%|██████████| 10000/10000 [00:19<00:00, 509.10it/s]


In [6]:
np.save('circle/new_X_train.npy', new_X_train)
np.save('circle/new_X_test.npy', new_X_test)
np.save('circle/new_X_test_45.npy', new_X_test_45)
np.save('circle/new_X_test_90.npy', new_X_test_90)

In [7]:
 np.save('circle/y_train.npy', y_train)
 np.save('circle/y_test.npy', y_test)


In [ ]:
model = get_model(new_X_train.shape[1:])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
for i in range(10):
    model.fit(new_X_train, y_train, batch_size=64, epochs=1)
    y_pred = model.predict(new_X_test).argmax(axis=1)
    test_acc_00 = np.mean(y_pred == y_test.ravel())*100
    y_pred = model.predict(new_X_test_45).argmax(axis=1)
    test_acc_45 = np.mean(y_pred == y_test.ravel())*100
    y_pred = model.predict(new_X_test_90).argmax(axis=1)
    test_acc_90 = np.mean(y_pred == y_test.ravel())*100
    print(f'A00={test_acc_00:.2f} A45={test_acc_45:.2f} A90={test_acc_90:.2f}')

In [ ]:
new_X_test_rot = []
for x in tqdm(X_test):
    xr = rotate(x, np.random.randint(-90, 90))
    new_X_test_rot.append(convert_to_lossless_polar(xr))
new_X_test_rot = np.array(new_X_test_rot)

In [ ]:
y_pred = model.predict(new_X_test_rot[:, offset:])

In [ ]:
np.mean(y_pred.argmax(axis=1) == y_test)*100